In [2]:
import json 
import requests
import pandas as pd

The following are the unique api keys that I alternated between to pull the data as there were hourly request limits.

In [22]:
api_key_1 = 'eH8Q9IhlhrVghuEXjqZgNdxQ30zUEylwaGSPRQtq'
api_key_2 = 'JkenCwSqw0QGJb3HGwP8JIYQ9bKhGQEVVxGjfbm5'
headers = {'Content-Type': 'application/json'}

With the below code, I got all the location and ID information for the comments

In [8]:
# initialize list to collect comment IDs
comm_ids = []

# for each page, get the comment id
for pg in range(20):
    url= 'https://api.regulations.gov/v4/comments?filter[commentOnId]=0900006484f67412&page[size]=250&page[number]='+str(pg)+'&api_key='+api_key
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        doc_dict = json.loads(response.content.decode('utf-8'))  
        #docDF = pd.json_normalize(doc_dict)
        #get all comment ids for the page and append to list
        comm_dict = doc_dict['data']
        for comms in comm_dict:
            if comms['type'] == 'comments':
                comm_ids.append(comms['id'])

The below initializes a blank DataFrame. Due to the request limits for the API key, I had to pull about 5,000 records once per hour. Therefore, I would append each new 5,000 records to the original dataFrame. I re-ran the the second code chunk manually, so this may give different results if you re-run it

In [ ]:
df_total = pd.DataFrame(columns = ['ID', 'Num of Duplicates', 'Comment', 'Page Count'])

In [48]:
comm_df = pd.DataFrame(columns = ['ID', 'Num of Duplicates', 'Comment', 'Page Count'])
# last ran at 2:46

# for each comment, get all of its associated information
for c in comm_ids[4500:]: # pre-set up; change this
    url= 'https://api.regulations.gov/v4/comments/'+c+'?api_key='+api_key_1 #change before running
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
        comm_info = json.loads(response.content.decode('utf-8')) 
        comm_dict_info = {'ID': comm_info['data']['id'], 
               'Num of Duplicates': comm_info['data']['attributes']['duplicateComments'],
               'Comment': comm_info['data']['attributes']['comment'],
               'Page Count': comm_info['data']['attributes']['pageCount']}
        comm_df = comm_df.append(comm_dict_info, ignore_index = True)
df_total = df_total.append(comm_df, ignore_index = True)

From the below output, we can see that there are 4,500 comments

In [49]:
df_total

,ID,Num of Duplicates,Comment,Page Count
0,CDC-2022-0024-1515,0,"Good Afternoon,<br/><span style='padding-left:...",1
1,CDC-2022-0024-1497,0,There is a real concern about the abuse of opi...,1
2,CDC-2022-0024-1501,0,I HAVE BEEN A PATIENT AT [location redacted] I...,1
3,CDC-2022-0024-1504,0,<span style='padding-left: 30px'></span>Sworn ...,1
4,CDC-2022-0024-1502,0,"My name is [name redacted]. I am 66 years old,...",1
...,...,...,...,...
4496,CDC-2022-0024-1726,0,I am a retired physician. I am disabled. I als...,1
4497,CDC-2022-0024-1729,0,I&#39;ve suffered from chronic pain since 2006...,1
4498,CDC-2022-0024-1727,0,I apologize for a few typos in my first commen...,1
4499,CDC-2022-0024-1742,0,I have been a chronic pain patient for 21 year...,1


In [50]:
# convert the file to Excel
df_total.to_excel('Opioid Comment Data from API.xlsx')

C:\Users\cbolt\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1168: UserWarning: Ignoring URL 'https://www.federalregister.gov/documents/2021/12/02/2021-26227/established-aggregate-production-quotas-for-schedule-i-and-ii-controlled-substances-and-assessment%3cbr/%3e%3cbr/%3eThe%20DEA%20and%20FDA%20NEVER%20SET%20STRICT%20MME%20requirements%20on%20the%20American%20people,%20If%20you%20read%20the%20federal%20register%20from%2012/02/21.%20%3cbr/%3e%3cbr/%3ePrescribing%20Hesitancy%3cbr/%3e%3cbr/%3eIssue:%3cbr/%3e%3cbr/%3eMany%20commenters,%20most%20of%20whom%20self-identified%20as%20chronic%20pain%20patients,%20expressed%20general%20concerns%20that%20the%20%3cbr/%3e%3cbr/%3eCDC%20Guidelines%20for%20Prescribing%20Opioids%20for%20Chronic%20Pain,%3cbr/%3e%3cbr/%3eissued%20in%202016,%20are%20preventing%20doctors%20from%20prescribing%20pain%20medication%20in%20dosages%20that%20adequately%20control%20chronic%20pain,%20forcing%20them%20to%20taper%20opioid%20medication%20dosages%20inappropriat